In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd '/content/drive/MyDrive/ClassAug Work/outlier-exposure'

/content/drive/.shortcut-targets-by-id/12PFZZk88jXqRuPPPvsCgycke0gd7aQBI/ClassAug Work/outlier-exposure


In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import os
import pickle
import argparse
import time
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torchvision.transforms as trn
import torchvision.datasets as dset
from torch.utils.data import Dataset
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.model_selection import train_test_split

if __package__ is None:
    import sys
    from os import path

    from utils.validation_dataset import validation_split

In [ ]:
batch_size=128
calibration=False
dataset='twitter_us_airline_sentiment'
decay=0.0005
droprate=0.3
epochs=1000
layers=40
learning_rate=0.2
load=''
model='allconv'
momentum=0.9
ngpu=1
oe_batch_size=256
prefetch=0
save='./snapshots/oe_scratch_new'
test=False
test_bs=200
widen_factor=2

In [ ]:
print(torch.cuda.get_device_name(0))

Tesla T4


In [ ]:
class IDDataset(Dataset):
  def __init__(self, embs, labels):
    self.embs = torch.from_numpy(embs).float()
    self.labels = torch.from_numpy(np.squeeze(labels)).to(dtype=torch.long)

  def __len__(self):
    return self.embs.shape[0]

  def __getitem__(self, idx):
    return self.embs[idx], self.labels[idx]

class OODDataset(Dataset):
  def __init__(self, embs):
    self.embs = torch.from_numpy(embs).float()

  def __len__(self):
    return self.embs.shape[0]

  def __getitem__(self, idx):
    return self.embs[idx], 0

In [ ]:
class OEModel(nn.Module):
    def __init__(self, num_classes):
        super(OEModel, self).__init__()

        self.l1 = nn.Linear(512, 512)
        self.d1 = nn.Dropout(0.3)
        self.classifier = nn.Linear(512, num_classes)
        self.softmax = nn.Softmax()

    def forward(self, x):
        x = self.l1(x)
        x = self.d1(x)
        x = self.classifier(x)
        x = self.softmax(x)
        return x

In [ ]:
state = {'batch_size': 128, 'calibration': False, 'dataset': 'cifar10', 'decay': 0.0005, 'droprate': 0.3, 'epochs': 100, 'layers': 40, 'learning_rate': 0.1, 'load': '', 'model': 'allconv', 'momentum': 0.9, 'ngpu': 0, 'oe_batch_size': 256, 'prefetch': 0, 'save': './snapshots/oe_scratch', 'test': False, 'test_bs': 200, 'widen_factor': 2}

torch.manual_seed(1)
np.random.seed(1)

# mean and standard deviation of channels of CIFAR-10 images
mean = [x / 255 for x in [125.3, 123.0, 113.9]]
std = [x / 255 for x in [63.0, 62.1, 66.7]]

train_transform = trn.Compose([trn.RandomHorizontalFlip(), trn.RandomCrop(32, padding=4),
                               trn.ToTensor(), trn.Normalize(mean, std)])
test_transform = trn.Compose([trn.ToTensor(), trn.Normalize(mean, std)])

embs_data = None
with open('../embeddings/twitter_us_airline_sentiment_embeddings.pkl', 'rb') as f:
    embs_data = pickle.load(f)
embs = embs_data['embs']
labels = embs_data['labels']
x_train, x_test, y_train, y_test = train_test_split(embs, labels, test_size=0.30, random_state=43, stratify=labels)
train_data_in = IDDataset(x_train, y_train)
test_data = IDDataset(x_test, y_test)
num_classes = 3


calib_indicator = ''
if calibration:
    train_data_in, val_data = validation_split(train_data_in, val_share=0.1)
    calib_indicator = '_calib'

embs_data = None
with open('../embeddings/sst_embeddings.pkl', 'rb') as f:
    embs_data = pickle.load(f)
ood_data = OODDataset(embs_data['embs'])

train_loader_in = torch.utils.data.DataLoader(
    train_data_in,
    batch_size=batch_size, shuffle=True,
    num_workers=prefetch, pin_memory=True)

train_loader_out = torch.utils.data.DataLoader(
    ood_data,
    batch_size=oe_batch_size, shuffle=False,
    num_workers=prefetch, pin_memory=True)

test_loader = torch.utils.data.DataLoader(
    test_data,
    batch_size=batch_size, shuffle=False,
    num_workers=prefetch, pin_memory=True)

net = OEModel(num_classes)


start_epoch = 0
# Restore model if desired
if load != '':
    for i in range(1000 - 1, -1, -1):
        model_name = os.path.join(save, dataset + calib_indicator + '_' + model +
                              '_oe_scratch_epoch_' + str(i) + '.pt')
        if os.path.isfile(model_name):
            net.load_state_dict(torch.load(model_name))
            print('Model restored! Epoch:', i)
            start_epoch = i + 1
            break
    if start_epoch == 0:
        assert False, "could not resume"

if ngpu > 1:
    net = torch.nn.DataParallel(net, device_ids=list(range(ngpu)))

if ngpu > 0:
    net.cuda()
    torch.cuda.manual_seed(1)

cudnn.benchmark = True  # fire on all cylinders

optimizer = torch.optim.SGD(net.parameters(), state['learning_rate'], momentum=state['momentum'], weight_decay=state['decay'], nesterov=True)

def cosine_annealing(step, total_steps, lr_max, lr_min):
    return lr_min + (lr_max - lr_min) * 0.5 * (
            1 + np.cos(step / total_steps * np.pi))


scheduler = torch.optim.lr_scheduler.LambdaLR(
    optimizer,
    lr_lambda=lambda step: cosine_annealing(
        step,
        epochs * len(train_loader_in),
        1,  # since lr_lambda computes multiplicative factor
        1e-6 / learning_rate))


# /////////////// Training ///////////////

def train():
    net.train()  # enter train mode
    loss_avg = 0.0
    correct = 0
    # start at a random point of the outlier dataset; this induces more randomness without obliterating locality
    train_loader_out.dataset.offset = np.random.randint(len(train_loader_out.dataset))
    for in_set, out_set in zip(train_loader_in, train_loader_out):
        data = torch.cat((in_set[0], out_set[0]), 0)
        target = in_set[1]

        data, target = data.cuda(), target.cuda()

        # forward
        x = net(data)

        # backward
        scheduler.step()
        optimizer.zero_grad()

        loss = F.cross_entropy(x[:len(in_set[0])], target)
        # cross-entropy from softmax distribution to uniform distribution
        loss += 0.5 * -(x[len(in_set[0]):].mean(1) - torch.logsumexp(x[len(in_set[0]):], dim=1)).mean()

        loss.backward()
        optimizer.step()

        # exponential moving average
        loss_avg = loss_avg * 0.8 + float(loss) * 0.2

    state['train_loss'] = loss_avg

# test function
def test():
    net.eval()
    loss_avg = 0.0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.cuda(), target.cuda()

            # forward
            output = net(data)
            loss = F.cross_entropy(output, target)

            # accuracy
            pred = output.data.max(1)[1]
            correct += pred.eq(target.data).sum().item()

            # test loss average
            loss_avg += float(loss.data)

    state['test_loss'] = loss_avg / len(test_loader)
    state['test_accuracy'] = correct / len(test_loader.dataset)


if test:
    test()
    print(state)
    exit()

# Make save directory
if not os.path.exists(save):
    os.makedirs(save)
if not os.path.isdir(save):
    raise Exception('%s is not a dir' % save)

with open(os.path.join(save, dataset + calib_indicator + '_' + model +
                                  '_oe_scratch_training_results.csv'), 'w') as f:
    f.write('epoch,time(s),train_loss,test_loss,test_error(%)\n')

print('Beginning Training\n')
best_acc = 0
# Main loop
for epoch in range(start_epoch, epochs+start_epoch):
    state['epoch'] = epoch

    begin_epoch = time.time()

    train()
    test()

    if 'test_accuracy' in state and best_acc < state['test_accuracy']:
      best_acc = state['test_accuracy']
      # Save model
      torch.save(net.state_dict(),
                os.path.join(save, dataset + calib_indicator + '_' + model +
                              '_oe_scratch_epoch_' + str(epoch) + '.pt'))
      # Let us not waste space and delete the previous model
      prev_path = os.path.join(save, dataset + calib_indicator + '_' + model +
                              '_oe_scratch_epoch_' + str(epoch - 1) + '.pt')
      if os.path.exists(prev_path): os.remove(prev_path)

    # Show results

    with open(os.path.join(save, dataset + calib_indicator + '_' + model +
                                      '_oe_scratch_training_results_512h1.csv'), 'a') as f:
        f.write('%03d,%05d,%0.6f,%0.5f,%0.2f\n' % (
            (epoch + 1),
            time.time() - begin_epoch,
            state['train_loss'],
            state['test_loss'],
            100 - 100. * state['test_accuracy'],
        ))

    # # print state with rounded decimals
    # print({k: round(v, 4) if isinstance(v, float) else v for k, v in state.items()})

    print('Epoch {0:3d} | Time {1:5d} | Train Loss {2:.4f} | Test Loss {3:.3f} | Test Accuracy {4:.2f} | Test Error {5:.2f}'.format(
        (epoch + 1),
        int(time.time() - begin_epoch),
        state['train_loss'],
        state['test_loss'],
        state['test_accuracy'],
        100 - 100. * state['test_accuracy'])
    )


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  
/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


{'batch_size': 128, 'calibration': False, 'dataset': 'cifar10', 'decay': 0.0005, 'droprate': 0.3, 'epochs': 100, 'layers': 40, 'learning_rate': 0.1, 'load': '', 'model': 'allconv', 'momentum': 0.9, 'ngpu': 0, 'oe_batch_size': 256, 'prefetch': 0, 'save': './snapshots/oe_scratch', 'test': False, 'test_bs': 200, 'widen_factor': 2, 'test_loss': 1.1016003574643816, 'test_accuracy': 0.20651183970856102}
Beginning Training

Epoch   1 | Time     0 | Train Loss 1.5126 | Test Loss 0.931 | Test Accuracy 0.63 | Test Error 37.32
Epoch   2 | Time     0 | Train Loss 1.4696 | Test Loss 0.909 | Test Accuracy 0.63 | Test Error 37.30
Epoch   3 | Time     0 | Train Loss 1.4335 | Test Loss 0.874 | Test Accuracy 0.66 | Test Error 33.63
Epoch   4 | Time     0 | Train Loss 1.3901 | Test Loss 0.849 | Test Accuracy 0.69 | Test Error 30.53
Epoch   5 | Time     0 | Train Loss 1.4030 | Test Loss 0.833 | Test Accuracy 0.74 | Test Error 26.37
Epoch   6 | Time     0 | Train Loss 1.3827 | Test Loss 0.821 | Test Accura